# YOLO Local Inference API

This API allows you to run object detection on images using a trained YOLO model and return **annotated images** with bounding boxes and class labels. It works **locally** and can be called from any programming language or framework that can send HTTP requests.

---

## 1️⃣ Setup

### Install dependencies

```bash
pip install ultralytics fastapi uvicorn python-multipart pillow
```

### Save your trained YOLO weights

Place your trained YOLO weights in the same folder as the API:

```
best.pt
```

---

## 2️⃣ API Code

Create a file `api.py`:

```python
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import StreamingResponse
from ultralytics import YOLO
from io import BytesIO
from PIL import Image

# Load YOLO model
model = YOLO("best.pt")

app = FastAPI(title="YOLO Local API")

@app.post("/predict_image")
async def predict_image(file: UploadFile = File(...)):
    # Load uploaded image
    image = Image.open(BytesIO(await file.read())).convert("RGB")
    
    # Run prediction
    results = model.predict(source=image, conf=0.25, verbose=False)

    # Use Ultralytics built-in plotting (nice bounding boxes)
    result_img = results[0].plot()  # BGR numpy
    result_img = Image.fromarray(result_img[..., ::-1])  # BGR → RGB

    # Convert to bytes for streaming
    buf = BytesIO()
    result_img.save(buf, format="JPEG")
    buf.seek(0)

    return StreamingResponse(buf, media_type="image/jpeg")

@app.get("/")
def root():
    return {"message": "YOLO API running. POST an image to /predict_image"}
```

---

## 3️⃣ Run the API

```bash
uvicorn api:app --host 0.0.0.0 --port 8000
```

- API will run locally at: `http://localhost:8000/`
- Test root endpoint: `GET /` → confirms API is running.
- Predict endpoint: `POST /predict_image` with image file.

---

## 4️⃣ Using the API

### Python Example

```python
import requests
from PIL import Image
from io import BytesIO

url = "http://localhost:8000/predict_image"
files = {"file": open("test.jpg", "rb")}
response = requests.post(url, files=files)

# Open returned image
img = Image.open(BytesIO(response.content))
img.show()
```

### Notes

- The API **returns a JPEG image** with bounding boxes and class labels drawn.  
- Can be called from **any language or framework** that supports HTTP POST and file upload (JavaScript, C#, Java, etc.).  

---

## 5️⃣ Using CURL

```bash
curl -X POST "http://localhost:8000/predict_image" \
  -F "file=@test.jpg" \
  --output result.jpg
```

- `result.jpg` will contain the annotated image.

---

## 6️⃣ Tips

- Only works **locally** unless you expose your machine on the network.  
- Adjust confidence threshold in the API (`conf=0.25`) if needed.  
- Can be easily integrated with any local frontend or framework (Streamlit, Tkinter, PyQt, Electron, etc.).
